# Read cone data and create Target

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import h5py

In [ ]:
import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', '_pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import math_utils
import data_utils

# Get Cone Release

In [ ]:
# Load Ground Truth data.
RawData_file = os.path.join('RawData', 'ConeData_ReleaseData.h5')
with h5py.File(RawData_file, 'r') as RawData:    

    dict_data = {}
    for k, v in RawData.items():
        print(k)
        dict_data[k] = np.array(v)
        print(dict_data[k].shape)

In [ ]:
# Data to dicts.
cone0 = {}
cone0['mean']   = dict_data['Averages0'][:,0]
cone0['traces'] = dict_data['Snippets0'][:,:,0]
cone0['time']   = dict_data['SnippetsTimes0'][:,:,0]

print(cone0['mean'].shape, cone0['traces'].shape, cone0['time'].shape)

cone1 = {}
cone1['mean']   = dict_data['Averages0'][:,1]
cone1['traces'] = dict_data['Snippets0'][:,:,1]
cone1['time']   = dict_data['SnippetsTimes0'][:,:,1]


print(cone1['mean'].shape, cone1['traces'].shape, cone1['time'].shape)

In [ ]:
def plot_cone_date(cone_data):
    fig, axs = plt.subplots(2,1,figsize=(15,8))
    axs[0].plot(cone_data['time']-cone_data['time'][0,:], cone_data['traces'], c='k', lw=0.3)
    axs[0].plot(
        cone_data['time'][:,0]-cone_data['time'][0,0],
        np.mean(cone_data['traces'], axis=1), c='b', lw=1
    )
    axs[0].fill_between(
        cone_data['time'][:,0]-cone_data['time'][0,0],
        np.mean(cone_data['traces'], axis=1) - np.std(cone_data['traces'], axis=1),
        np.mean(cone_data['traces'], axis=1) + np.std(cone_data['traces'], axis=1),
        color='r', alpha=0.5
    )
    axs[1].plot(
        cone_data['time'][:,0]-cone_data['time'][0,0],
        np.mean(cone_data['traces'], axis=1), c='b', lw=1
    )
    axs[1].plot(
        cone_data['time'][:,0]-cone_data['time'][0,0], cone_data['mean'], 'r--', lw=1
    )

In [ ]:
plot_cone_date(cone0)

In [ ]:
plot_cone_date(cone1)

In [ ]:
cones = [cone0, cone1]

In [ ]:
# Get minium and maximum range according to time points.
t_rng = [np.max(cone['time'][:,idx]) - np.min(cone['time'][:,idx]) for cone in cones for idx in range(cone['time'].shape[1])]
print(np.min(t_rng), np.max(t_rng), np.max(t_rng)-np.min(t_rng))

In [ ]:
# Set release time.
t_release = np.linspace(0.016, 33.032, cone0['mean'].shape[0])
t_release

# Load stimulus

In [ ]:
# Save.
stim_file = os.path.join('PreprocessedData', 'ConeData_stimulus_time_and_amp_corrected.csv')
cone_data_amp_and_time_corrected = pd.read_csv(stim_file)

In [ ]:
def plot_stim_vs_response(cone_data):
    xlims = [(4.6, 5.8), (1.5, 2.5), (7.5, 8.5), (29.5, 30.5)]
    fig, axs = plt.subplots(len(xlims),1,figsize=(15,len(xlims)*4))
    for ax, xlim in zip(axs, xlims):
        ax.set_title('Cone 0')
        for idx in range(cone_data['time'].shape[1]):
            ax.plot(t_release, cone_data['traces'][:,idx], alpha=0.3, c='k')
        ax.plot(t_release, np.mean(cone_data['traces'][:,:], axis=1), alpha=1, c='r')
        ax.set_xlim(xlim)
        ax.set_ylim(-2.6, 1.3)
        
        ax2 = ax.twinx()
        ax2.plot(cone_data_amp_and_time_corrected['Time'], cone_data_amp_and_time_corrected['Stim'], c='b')
        ax2.set_ylim(-0.1,1.1)

In [ ]:
plot_stim_vs_response(cone0)

In [ ]:
plot_stim_vs_response(cone1)

## Align cones

In [ ]:
from scipy.optimize import minimize

norm_mean0 = math_utils.normalize(cone0['mean'])
norm_mean1 = math_utils.normalize(cone1['mean'])

In [ ]:
# Optimization helper functions.
def trace_loss(trace, target):
    '''Mean Squared Error between a trace and a target trace.
    '''
    return np.mean((trace - target)**2)

def lin_trans(params, trace):
    '''Linear transformation of a trace.
    '''
    assert params.size == 2
    return params[0]+params[1]*trace

def lin_trans_loss(params, trace, target):
    '''Performs linear transformation and computes the loss.
    '''
    return trace_loss(lin_trans(params, trace), target)
  
def best_lin_trans(trace, target):
    '''Find best linear transformation to get smallest loss. 
    '''

    # Optimize parameters.
    n_iter = 100
    losses = np.ones(n_iter)*np.inf
    best_params = np.zeros((n_iter,2))
    
    for i in range(n_iter):
        x_opt = minimize(lin_trans_loss, x0=[np.random.normal(0,1), np.random.normal(1,1)],\
                         args=(trace, target), method='SLSQP', bounds=((-1,1), (-1,1)))
        losses[i] = x_opt.fun
        best_params[i,:] = x_opt.x
    
    # Extract best result. Normalize loss.
    best_it = np.nanargmin(losses)
    best_loss = losses[best_it]
      
    best_trace = lin_trans(best_params[best_it, :], trace)
    
    print(best_params[best_it, :])
    
    return best_trace, best_loss

In [ ]:
# Optimize.
np.random.seed(45687556)

norm_mean0_shifted, best_loss = best_lin_trans(norm_mean0, target=norm_mean1)
print(best_loss)

norm_mean = 0.5*(norm_mean1 + norm_mean0_shifted)

In [ ]:
# Plot.
plt.figure(figsize=(15,2))
plt.plot(t_release, norm_mean1, label='norm cone 1')
plt.plot(t_release, norm_mean0_shifted, label='norm cone 0 aligned')
plt.plot(t_release, norm_mean, label='norm mean')
plt.legend()
plt.show()

In [ ]:
# Create DataFrame.
cone_data_release = pd.DataFrame({'Time': t_release, 'mean': math_utils.normalize(norm_mean)})

In [ ]:
# Plot raw trace.
plt.figure(figsize=(15,6))
plt.plot(cone_data_release['Time'], cone_data_release['mean'])
plt.show()

In [ ]:
# Save.
release_file = os.path.join('PreprocessedData', 'ConeData_ReleaseMeanData.csv')
cone_data_release.to_csv(release_file, index=False)

# Export data

In [ ]:
cone_df_export = pd.DataFrame(
    np.vstack([cone_data_release['Time'], cone0['traces'].T, cone1['traces'].T]).T,
    columns=['Time'] + ['Cone1_rec'+str(i+1) for i in range(5)] + ['Cone2_rec'+str(i+1) for i in range(5)]
)

cone_df_export['Cone1_mean'] = cone0['mean']
cone_df_export['Cone2_mean'] = cone1['mean']

cone_df_export['Cone1_mean_aligned'] = norm_mean0_shifted
cone_df_export['Cone2_mean_aligned'] = norm_mean1
cone_df_export['Cones_mean_of_means'] = norm_mean

assert np.allclose(cone_df_export.loc[:,['Cone1_rec'+str(i+1) for i in range(5)]].mean(axis=1),
                   cone_df_export.loc[:,'Cone1_mean'], atol=1e-4, rtol=1e-4)

In [ ]:
cone_df_export.head()

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,3))
axs[0].plot(cone_df_export['Cone1_mean'], c='r', alpha=0.3)
axs[1].plot(cone_df_export['Cone2_mean'], c='b', alpha=0.3)

axs[0].twinx().plot(cone_df_export['Cone1_mean_aligned'], c='r', ls='--')
axs[1].twinx().plot(cone_df_export['Cone2_mean_aligned'], c='b', ls='--')

In [ ]:
plt.plot(0.5*(cone_df_export['Cone1_mean_aligned'] + cone_df_export['Cone2_mean_aligned']))
plt.plot(cone_df_export['Cones_mean_of_means'], ls=':')

In [ ]:
data_utils.save_var(cone_df_export, 'PreprocessedData/iGluSnFR_traces_Cones.pkl')